# Data processing in preparation for plotting

- Raw CERES and ERA5 data is prepared for figure creation
- Loopiness is calculated 

In [5]:
import climlab
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from IPython.display import clear_output
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as patches
from matplotlib.colors import LogNorm
import matplotlib.colors
import matplotlib as mpl

### Useful Functions

In [2]:
def progress(progress):
    """
    A loading bar 
    
    Parameters:
        Float in range 0 to 1 as fractional progress
    
    Returns:
        None, prints loading bar
    """

    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait=True)
    text = "Progress: [{0}] {1:.1f}%".format(
        "#" * block + "-" * (bar_length - block), progress * 100
    )
    print(text)

def trap_area(x,y):
    # Calculates the area of a loop with the trapezium rule 
    # Note the connection of the last to the first points
    area = np.trapz(y, x) + np.trapz([y[-1], y[0]], [x[-1], x[0]])
    return area

# CERES OLR 2001-2020 with ERA5 T2m 2001-2020 for Hyst 

In [3]:
# For interp_like 
# An ERA5 file on a 1x1 grid
data_reference = xr.open_dataset('../../Data/ERA5/olr_olrClr.1x1deg.yr1980-1999.nc').mean(dim="time").rename({'longitude':'lon', 'latitude':'lat'})

# CERES OLR + ERA5 T2m for 2000-2020
data_olr_raw = xr.open_dataset('../../Data/CERES/CERES_EBAF-TOA_Ed4.1_Subset_200003-202012.nc').sel(time=slice("2001-01-01", "2020-12-30"))
data_ts_raw = xr.open_dataset('../../Data/ERA5/t2m_2000_2020.nc').rename({'latitude': 'lat', 'longitude': 'lon'}).mean(dim='expver').sel(time=slice("2001-01-01", "2020-12-30"))

# Interp like old data
data_olr = data_olr_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})
data_ts = data_ts_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})

# Average by month
data_olr = data_olr.groupby("time.month").mean()
data_ts = data_ts.groupby("time.month").mean()

# Merge into single dataset
data = xr.merge([data_olr, data_ts])

## Clearsky

In [6]:
grad_vals = np.zeros_like(data.mean(dim='month').t2m.values)
r2_vals = np.zeros_like(data.mean(dim='month').t2m.values)
hyst_vals = np.zeros_like(data.mean(dim='month').t2m.values)
hyst_over_olr_vals = np.zeros_like(data.mean(dim='month').t2m.values)

lats = data.lat.values
lons = data.lon.values

In [8]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).t2m.values, data.sel(lat=lats[i], lon=lons[j]).toa_lw_clr_c_mon.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).toa_lw_clr_c_mon.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).t2m.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).t2m.values, data.sel(lat=lats[i],lon=lons[j]).toa_lw_clr_c_mon.values)
        hyst_val = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).t2m.values) - min(data.sel(lat=lats[i],lon=lons[j]).t2m.values))
        hyst_vals[i,j] = hyst_val
        
        olr_range = max(data.sel(lat=lats[i],lon=lons[j]).toa_lw_clr_c_mon.values) - min(data.sel(lat=lats[i],lon=lons[j]).toa_lw_clr_c_mon.values)
        hyst_over_olr_vals[i,j] = hyst_val * 100 / olr_range
progress(1)

Progress: [####################] 100.0%


In [9]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)
data['hyst_over_olr_range'] = (('lat', 'lon'), hyst_over_olr_vals)

In [11]:
data.to_netcdf('../../Data/CERES/clear_sky_ceres.nc')

## Allsky

In [12]:
grad_vals = np.zeros_like(data.mean(dim='month').t2m.values)
r2_vals = np.zeros_like(data.mean(dim='month').t2m.values)
hyst_vals = np.zeros_like(data.mean(dim='month').t2m.values)
hyst_over_olr_vals = np.zeros_like(data.mean(dim='month').t2m.values)

lats = data.lat.values
lons = data.lon.values

In [ ]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).t2m.values, data.sel(lat=lats[i], lon=lons[j]).toa_lw_all_mon.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).toa_lw_all_mon.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).t2m.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).t2m.values, data.sel(lat=lats[i],lon=lons[j]).toa_lw_all_mon.values)
        hyst_val = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).t2m.values) - min(data.sel(lat=lats[i],lon=lons[j]).t2m.values))
        hyst_vals[i,j] = hyst_val 
        
        olr_range = max(data.sel(lat=lats[i],lon=lons[j]).toa_lw_all_mon.values) - min(data.sel(lat=lats[i],lon=lons[j]).toa_lw_all_mon.values)
        hyst_over_olr_vals[i,j] = hyst_val * 100 / olr_range
        print(hyst_val, olr_range, hyst_over_olr_vals[i,j])
progress(1)

In [14]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)
data['hyst_over_olr_range'] = (('lat', 'lon'), hyst_over_olr_vals)

In [15]:
data.to_netcdf('../../Data/CERES/all_sky_ceres.nc')

# ERA5 OLR for 2000 - 2020 with ERA5 T2m for Loopiness calculations

In [46]:
# For interp_like 
data_reference = xr.open_dataset('../../Data/ERA5/olr_olrClr.1x1deg.yr1980-1999.nc').mean(dim="time").rename({'longitude':'lon', 'latitude':'lat'})

# Import data
data_olr_raw = xr.open_dataset('../../Data/ERA5/era5_olr_2000-20020.nc').mean(dim="realization").sel(time=slice("2001-01-01", "2020-12-30"))
data_ts_raw = xr.open_dataset('../../Data/ERA5/t2m_2000_2020.nc').rename({'latitude': 'lat', 'longitude': 'lon'}).mean(dim='expver').sel(time=slice("2001-01-01", "2020-12-30"))

# Correcting for data being integrated over 24 hours, and correcting sign
data_olr_raw['rlt_accumulated'] = data_olr_raw['rlt_accumulated'] / -86400 
data_olr_raw['ttrc_NON_CDM'] = data_olr_raw['ttrc_NON_CDM'] / -86400 

# Interp_like old dataset
data_olr = data_olr_raw.assign_coords(lon=((data_olr_raw.lon + 360) % 360)).interp_like(data_reference)
data_ts = data_ts_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})

# Average by month 
data_olr = data_olr.groupby("time.month").mean()
data_ts = data_ts.groupby("time.month").mean()

# Merge into a single dataset 
data = xr.merge([data_olr, data_ts])

In [4]:
grad_vals = np.zeros_like(data.mean(dim='month').t2m.values)
r2_vals = np.zeros_like(data.mean(dim='month').t2m.values)
hyst_vals = np.zeros_like(data.mean(dim='month').t2m.values)

lats = data.lat.values
lons = data.lon.values

In [5]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).t2m.values, data.sel(lat=lats[i], lon=lons[j]).ttrc_NON_CDM.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).ttrc_NON_CDM.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).t2m.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).t2m.values, data.sel(lat=lats[i],lon=lons[j]).ttrc_NON_CDM.values)
        hyst_vals[i,j] = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).t2m.values) - min(data.sel(lat=lats[i],lon=lons[j]).t2m.values))
progress(1)

Progress: [####################] 100.0%


In [6]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)

In [7]:
data.to_netcdf('clear_sky_era5.nc')

# Cluster Output Calculated ERA5 OLR for 2000 - 2020 with ERA5 T2m for Hyst

In [3]:
# For interp_like 
data_reference = xr.open_dataset('../../Data/ERA5/olr_olrClr.1x1deg.yr1980-1999.nc').mean(dim="time").rename({'longitude':'lon', 'latitude':'lat'})

# Import data
data_raw = xr.open_dataset('../../Data/Cluster/Combined_data_ceres.nc')
data = data_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})

In [4]:
grad_vals = np.zeros_like(data.mean(dim='month').ts.values)
r2_vals = np.zeros_like(data.mean(dim='month').ts.values)
hyst_vals = np.zeros_like(data.mean(dim='month').ts.values)

lats = data.lat.values
lons = data.lon.values

In [8]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).ts.values, data.sel(lat=lats[i], lon=lons[j]).olr_calc.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).olr_calc.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).ts.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).ts.values, data.sel(lat=lats[i],lon=lons[j]).olr_calc.values)
        hyst_vals[i,j] = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).ts.values) - min(data.sel(lat=lats[i],lon=lons[j]).ts.values))
progress(1)

Progress: [####################] 100.0%


In [9]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)

In [10]:
data.to_netcdf('clear_sky_calculated.nc')

# Cluster Output Const T

In [21]:
# For interp_like 
data_reference = xr.open_dataset('../../Data/ERA5/olr_olrClr.1x1deg.yr1980-1999.nc').mean(dim="time").rename({'longitude':'lon', 'latitude':'lat'})

# Import data
data_raw = xr.open_dataset('../../Data/Cluster/Combined_data_ceres_const_t.nc')
data = data_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})

In [20]:
grad_vals = np.zeros_like(data.mean(dim='month').ts.values)
r2_vals = np.zeros_like(data.mean(dim='month').ts.values)
hyst_vals = np.zeros_like(data.mean(dim='month').ts.values)

lats = data.lat.values
lons = data.lon.values

In [17]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).ts.values, data.sel(lat=lats[i], lon=lons[j]).olr_calc.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).olr_calc.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).ts.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).ts.values, data.sel(lat=lats[i],lon=lons[j]).olr_calc.values)
        hyst_vals[i,j] = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).ts.values) - min(data.sel(lat=lats[i],lon=lons[j]).ts.values))
progress(1)

Progress: [####################] 100.0%


In [18]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)

In [19]:
data.to_netcdf('clear_sky_calculated_const_t.nc')

# Cluster Output Const RH

In [19]:
# For interp_like 
data_reference = xr.open_dataset('../../Data/ERA5/olr_olrClr.1x1deg.yr1980-1999.nc').mean(dim="time").rename({'longitude':'lon', 'latitude':'lat'})

# Import data
data_raw = xr.open_dataset('../../Data/Cluster/Combined_data_ceres_const_rh.nc')
data = data_raw.interp_like(data_reference, kwargs={"fill_value": "extrapolate"})

In [20]:
grad_vals = np.zeros_like(data.mean(dim='month').ts.values)
r2_vals = np.zeros_like(data.mean(dim='month').ts.values)
hyst_vals = np.zeros_like(data.mean(dim='month').ts.values)

lats = data.lat.values
lons = data.lon.values

In [21]:
for i in range(len(lats)):
    for j in range(len(lons)):
        p = np.polyfit(data.sel(lat=lats[i], lon=lons[j]).ts.values, data.sel(lat=lats[i], lon=lons[j]).olr_calc.values, 1)
        grad_vals[i,j] = p[0]
        
        y = data.sel(lat=lats[i], lon=lons[j]).olr_calc.values
        f = [p[0]*k + p[1] for k in data.sel(lat=lats[i], lon=lons[j]).ts.values]
        r2 = 1. - np.sum( (y-f)**2 )/np.sum( (y-np.mean(y))**2 )
        r2_vals[i,j] = r2
        
        t_area = trap_area(data.sel(lat=lats[i],lon=lons[j]).ts.values, data.sel(lat=lats[i],lon=lons[j]).olr_calc.values)
        hyst_vals[i,j] = t_area / (max(data.sel(lat=lats[i],lon=lons[j]).ts.values) - min(data.sel(lat=lats[i],lon=lons[j]).ts.values))
progress(1)

Progress: [####################] 100.0%


In [22]:
data['grad'] = (('lat', 'lon'), grad_vals)
data['r2'] = (('lat', 'lon'), r2_vals)
data['hyst'] = (('lat', 'lon'), hyst_vals)

In [23]:
data.to_netcdf('clear_sky_calculated_const_rh.nc')

# Processing Data for Fig. 4

In [120]:
data_t = xr.open_dataset('../../Data/Cluster/data_atm_t.nc')
data_r = xr.open_dataset('../../Data/Cluster/data_atm_r.nc')

data = xr.merge([data_t, data_r])

In [123]:
data.to_netcdf('../../Data/Cluster/data_atm.nc')